In [7]:
import torch
import numpy as np

In [104]:
file_base_path = "/global/cfs/cdirs/m3443/usr/xju/ITk/For2023Paper/metric_learning_testing/debug"
acorn_data_file = f"{file_base_path}/debug_data.pt"
data = torch.load(acorn_data_file, map_location="cpu")

/tmp/ipykernel_2283654/3261373381.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(acorn_data_file, map_location="cpu")


In [115]:
cpp_data_file = f"{file_base_path}/tracks.txt"
with open(cpp_data_file) as f:
    lines = f.readlines()

data_cpp = np.array(list(map(int, lines[0].split())), dtype=np.int64)

In [127]:
cpp_data_file2 = f"{file_base_path}/tracks_undirected_graph.txt"
with open(cpp_data_file2) as f:
    lines = f.readlines()

data_cpp2 = np.array(list(map(int, lines[0].split())), dtype=np.int64)

In [59]:
def get_track_arrays(in_array):
    tracks = []
    track = []
    for hit_id in in_array:
        if hit_id == -1:
            tracks.append(track)
            track = []
        else:
            track.append(hit_id.item())
    return tracks

In [128]:
tracks_from_acorn = get_track_arrays(data["track_candidates"])
tracks_from_cpp = get_track_arrays(data_cpp)
tracks_from_cpp2 = get_track_arrays(data_cpp2)

In [130]:
len(tracks_from_acorn), len(tracks_from_cpp), len(tracks_from_cpp2)

(4248, 4366, 4611)

In [109]:
tracks_from_acorn[0], tracks_from_cpp[0]

([14424, 1, 14508, 78, 14581, 14649, 21910, 21986, 22055, 22135, 62580],
 [1, 14508, 14581, 14649, 21910, 21986, 22055, 22135, 62580, 78])

In [138]:
def matching(tracks1, tracks2, threshold=0.7, min_common=999):
    matching_results = []
    only_in_1 = []
    for track1 in tracks1:
        matched = False
        for track2 in tracks2:
            track1_set = set(track1)
            track2_set = set(track2)
            common = track1_set & track2_set
            frac1 = len(common) / len(track1_set)
            frac2 = len(common) / len(track2_set)
            if ((frac1 > threshold and frac2 > threshold) or (len(common) >= min_common)
            ):
                matching_results.append((track1, track2, frac1, frac2))
                matched = True
        if not matched:
            only_in_1.append(track1)
    return matching_results, only_in_1

In [132]:
matched, only_in_acorn = matching(tracks_from_acorn, tracks_from_cpp, 0.7)
len(matched), len(matched) / len(tracks_from_acorn)

(3543, 0.8340395480225988)

In [137]:
matched, only_in_acorn2 = matching(tracks_from_acorn, tracks_from_cpp2, 0.7)
print(len(matched), len(matched) / len(tracks_from_acorn))

matched, only_in_cpp2 = matching(tracks_from_cpp2, tracks_from_acorn, 0.7)
print(len(matched), len(matched) / len(tracks_from_acorn))

4191 0.9865819209039548
4191 0.9865819209039548


In [119]:
def find_track_has_hit(tracks, hit_id):
    for track in tracks:
        if hit_id in track:
            return track
    return None

In [134]:
print(only_in_acorn[0])
print(find_track_has_hit(tracks_from_cpp, 165))

[2, 81, 165]
None


In [135]:
print(only_in_acorn[1])
print(find_track_has_hit(tracks_from_cpp, 84564))
print(find_track_has_hit(tracks_from_cpp, 1071))

[84564, 932, 15, 85, 1071, 163, 1156, 236, 1233, 309]
[84564, 932, 15, 85]
[1071, 1156, 1233, 163, 236, 309]


In [136]:
print(only_in_acorn2[0])
print(find_track_has_hit(tracks_from_cpp2, 1985))

[1985, 26158, 24774, 26223]
[1985, 24774, 24776, 24837, 26158, 26159, 26223, 26225, 26277]


In [142]:
print(only_in_acorn2[1])
print(find_track_has_hit(tracks_from_cpp2, 4532))
print(find_track_has_hit(tracks_from_cpp2, 3934))
print(find_track_has_hit(tracks_from_acorn, 83363))

[4532, 3804, 4601, 3874, 4672, 3934, 4731, 3983, 4034, 30092, 28989, 30155, 29055, 30212]
[4532, 3804, 4601, 3874, 4672]
[83363, 4542, 3810, 4606, 3877, 4673, 3934, 4731, 3983, 4034, 30092, 28989, 30155, 29055, 30212]
[83363, 4542, 3810, 4606, 3877, 4673]


In [145]:
data["track_candidates"]

tensor([ 14424,      1,  14508,  ..., 255907, 212432,     -1])

In [126]:
mask = data["gnn_edge_lists"][1] == 165
data["gnn_scores"][mask], data["gnn_edge_lists"][:, mask]

(tensor([6.7357e-02, 2.5866e-05, 2.5374e-05]),
 tensor([[ 81,  83,  86],
         [165, 165, 165]]))

[84564, 932, 15, 85]
[1071, 1156, 1233, 163, 236, 309]


In [100]:
for track in tracks_from_cpp:
    if 84564 in track:
        print(track)
        break

[84564, 932, 15, 85]


In [57]:
matched[0]

([1, 78, 14424, 14508, 14581, 14649, 21910, 21986, 22055, 22135, 62580],
 [1, 78, 14508, 14581, 14649, 21910, 21986, 22055, 22135],
 0.8181818181818182,
 1.0)